# Set Up

## Import Packages

In [85]:
# Lavorare con i dataframe
import pandas as pd
pd.set_option('display.max_columns', None) # Per visualizzare tutte le colonne di un dataset, display(df)
#pd.set_option('display.max_rows', None) # Per visualizzare tutte le righe di un dataset, display(df)
import numpy as np
from recordlinkage.preprocessing import clean
# Importare sys per indicare il path della cartella ove sono collocate le funzioni selfmade.
import sys
#sys.path.append('/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Funzioni')
from funzioni import alfabeto, pulisci_prezzo, pulisci_recensione, pulisci_distanza_centro, trasforma_in_data # Funzioni fatte da se


path_data_lake = "/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Data Lake"
path_data_product = "/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Data Product"

## Load the Data

Importare i dataframe contenenti informazioni su hotel e B&B riguardanti le città di Roma e Terni disponibili
sulle piattaforme di agoda e booking.     
Dati ottenuti tramite **web scraping**, per osservare il processo di scraping guardare al folder dedicato.

In [86]:
agoda_terni = pd.read_csv(f'{path_data_lake}/agoda_Terni.csv',index_col=0)
booking_terni = pd.read_csv(f'{path_data_lake}/booking_Terni.csv',index_col=0)

booking_roma = pd.read_csv(f'{path_data_lake}/booking_Roma.csv',index_col=0)
agoda_roma = pd.read_csv(f'{path_data_lake}/agoda_Roma.csv',index_col=0)

# Data overview

Osservare i dati per:
 - Capire se c'è possibilità di feature engineering;
 - Capire inoltre se ci sono delle variabili
che necessitano di pulizia.

## Dati riguardanti Agoda

Visualizzazione dati:

In [87]:
display(agoda_roma.head(1))
display(agoda_terni.head(1))

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Raeli Hotel Lazio,"7,7",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€159,"Stazione centrale Roma Termini, Roma - In pien...",3


,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0


Controllare che le variabili tra sul sito di **agoda** riguardo a Roma e Terni siano le stesse.

In [88]:
agoda_terni.columns.equals(agoda_roma.columns)

True

Le variabili tra agoda Roma e agoda Terni sono le stesse e sono:
   1. **Titolo** -> Nome della struttura;

   2. **Recensione voto numerico** -> Recensione con voto numerico;

   3. **Recensione voto parola** -> Recensione con voto a parola;

   4. **Numero recensioni** -> Numero di recensioni fatte alla struttura;

   5. **Date** -> Data di permanenza nella struttura;

   6. **Permanenza** -> Informazioni sulla permanenza, numero di notti e numero di persone (adulti o bambini);

   7. **Prezzo** -> Costo della permanenza;

   8. **Indirizzo** -> Indirizzo della struttura formato (città, provincia);

   9. **Valutazione** -> Le stelle di un hotel o la valutazione assegnata al B&B.

Sul dataset di agoda si può fare del feature engineering:

   - Creare variabile NUMERO NOTTI dalla variabile **permanenza**, deve essere un numero;

   - Creare variabile NUMERO PERSONE (adulte) e NUMERO PERSONE (bambini) dalla variabile **permanenza**;

   - Creare variabile DATA CHECK IN da **date**;

   - Creare variabile DATA CHECK OUT da **date**.



Per quanto riguarda la variabile **indirizzo**:
   - Nel dataframe **ROMA** ha il formato (zona, città - distanza centro)  es. Stazione centrale Roma Termini, Roma - In pieno centro;

   - Nel dataframe **TERNI** ha il formato (zona, città)  es. Terni, Terni;


Perciò per quanto riguarda Roma può anche essere creata la variabile **distanza dal centro**, oltre a **zona** e **città**.      
Invece riguarda Terni può essere creata la variabile **zona** e **città**.                  

Inoltre per effettuare le dovute analisi bisogna:
   - Ricodificare la variabile **prezzo** rimuovendo il simbolo dell'euro (accertarsi che i prezzi siano tutti in euro);

   - Sempre riguardo alla variabile **prezzo** eliminare il punto come separatore delle migliaia;

   - **numero recensioni** qualora comparisse "Novità su Agoda" inserire 0 (zero).

   - **numero recensioni** estrarre solo il numero di recensioni eliminado la stringa **recensioni** contenuta.

   - La variabile **distanza centro** andrà pulita e resa un numero (float).

## Dati riguardanti booking

Visualizzazione dati:

In [89]:
display(booking_roma.head(1))
display(booking_terni.head(1))

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...


,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,Hotel De Paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...


Controllare che le variabili tra sul sito di **booking** riguardo a Roma e Terni siano le stesse.

In [90]:
booking_terni.columns.equals(booking_roma.columns)

True

Le variabili tra agoda Roma e agoda Terni sono le stesse e sono:
   1. **Titolo** -> Nome della struttura;

   2. **Recensione voto numerico** -> Recensione con voto numerico;

   3. **Recensione voto parola** -> Recensione con voto a parola;

   4. **Numero recensioni** -> Numero di recensioni fatte alla struttura;

   5. **Date** -> Data di permanenza nella struttura;

   6. **Permanenza** -> Informazioni sulla permanenza, numero di notti e numero di personev(adulti o bambini);

   7. **Prezzo** -> Costo della permanenza;

   8. **Distanza centro** -> Distanza dal centro della città di riferimento; 

   9. **Indirizzo** -> Indirizzo della struttura formato (città, provincia);

   10. **Stelle** -> Stelle se la struttura è un hotel (NAN se non è hotel);

   11. **Valutazione booking** -> Valutazione assegnata al B&B (NAN se non è B&B);

   12. **Descrizione camera** -> Descrizione della camera, quante camere se c'è vista ecc...;

   13. **Configurazione camera** -> Se c'è il bagno, numero di letti ecc...;

   14. **Descrizione unprocessed** -> Descrizione camera + Configurazione camera.

Sul dataset di booking si può fare del feature engineering:
   - Creare variabile NUMERO NOTTI dalla variabile **permanenza**, deve essere un numero;

   - Creare variabile NUMERO PERSONE (adulte) e NUMERO PERSONE (bambini) dalla variabile **permanenza**;

   - Creare variabile DATA CHECK IN da **date**;

   - Creare variabile DATA CHECK OUT da **date**.

Per quanto riguarda la variabile **indirizzo**:
   - Nel dataframe **ROMA** ha il formato (zona, città)  es. Trionfale, Roma;

   - Nel dataframe **TERNI** ha il formato (città)  es. Terni;


Perciò per quanto riguarda Roma può anche essere creata la variabile **distanza dal centro**, oltre a **zona** e **città**.      
Invece riguarda Terni può essere creata la variabile **zona** e **città**.   

Inoltre per effettuare le dovute analisi bisogna:
   - Ricodificare la variabile **prezzo** rimuovendo il simbolo dell'euro (accertarsi che i prezzi siano tutti in euro);

   - Sempre riguardo alla variabile **prezzo** eliminare il punto come separatore delle migliaia;

   - **numero recensioni** qualora comparisse "Novità su booking" inserire 0 (zero).

   - **numero recensioni** estrarre solo il numero di recensioni eliminado la stringa **recensioni** contenuta.

   - La variabile **distanza centro** andrà pulita e resa un numero (float).

# Feature engineering

Creazione di nuove variabili partendo da quelle che si ha per estrarre informazioni aggiuntive.

## Feature engineering applicato ad Agoda

Queste trasformazioni saranno le stesse applicate al dataframe **agoda roma** e **agoda terni**. Si lavora su:
   - Numero notti;
   - Numero persone -> Numero adulti, numero bambini, totale ospiti;
   - Inizio permanenza;
   - Fine permanenza.
   
Per quanto riguarda le variabili da estrarre da **indirizzo** queste andranno trattate diversamente tra agoda ROMA e agoda TERNI:
   - Distanza centro;
   - Zona;
   - Città.



Già da questa fase di feature engineering si potrebbero unire i dataset riguardanti lo stesso sito web, per evitare errori sarà fatto nella fase di cleaning.

In [91]:
#pd.set_option('display.max_rows', 10)
agoda_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0


### Agoda Terni

In [92]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
agoda_terni["numero_notti"]=agoda_terni.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
agoda_terni["numero_persone"]=agoda_terni.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_terni["inizio_permanenza"]=agoda_terni.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_terni["fine_permanenza"]=agoda_terni.date.str.split("-").str[1]

#-----#
# Variabile da estrarre da indirizzo.

# Distanza centro, estratto dalla variabile indirizzo. Non è presente la distanza dal centro.
agoda_terni["distanza_centro"] = np.nan

# Zona, estratta dalla variabile indirizzo.
agoda_terni["zona"] = agoda_terni.indirizzo.str.split(",").str[0]

# Città, estratta dalla variabile indirizzo.
agoda_terni["città"] = agoda_terni.indirizzo.str.split(",").str[1]

In [93]:
agoda_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,distanza_centro,zona,città
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0,1 notte,2 adulti,1 agosto,2 agosto,NaN,Terni,Terni


### Agoda Roma

In [94]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
agoda_roma["numero_notti"]=agoda_roma.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
agoda_roma["numero_persone"]=agoda_roma.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_roma["inizio_permanenza"]=agoda_roma.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_roma["fine_permanenza"]=agoda_roma.date.str.split("-").str[1]

#-----#
# Variabile da estrarre da indirizzo.

# Distanza centro, estratto dalla variabile indirizzo.
agoda_roma["distanza_centro"] = agoda_roma.indirizzo.str.split("-").str[1]

# Zona, estratta dalla variabile indirizzo.
agoda_roma["zona"] = agoda_roma.indirizzo.str.split("-").str[0].str.split(",").str[0]

# Città, estratta dalla variabile indirizzo.
agoda_roma["città"] = agoda_roma.indirizzo.str.split("-").str[0].str.split(",").str[1]

In [95]:
agoda_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,distanza_centro,zona,città
0,Raeli Hotel Lazio,"7,7",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€159,"Stazione centrale Roma Termini, Roma - In pien...",3,1 notte,2 adulti,1 agosto,2 agosto,In pieno centro,Stazione centrale Roma Termini,Roma


## Feature engineering applicato a Booking

### Booking Terni

In [96]:
booking_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,Hotel De Paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...


In [97]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
booking_terni["numero_notti"]=booking_terni.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
booking_terni["numero_persone"]=booking_terni.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_terni["inizio_permanenza"]=booking_terni.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_terni["fine_permanenza"]=booking_terni.date.str.split("-").str[1]

# Zona, estratta dalla variabile indirizzo.
booking_terni["zona"] = booking_terni.indirizzo

# Città, estratta dalla variabile indirizzo.
booking_terni["città"] = booking_terni.indirizzo

In [98]:
booking_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,zona,città
0,Hotel De Paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...,1 notte,2 adulti,1 agosto,2 agosto,Terni,Terni


### Booking Roma

In [99]:
booking_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...


In [100]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
booking_roma["numero_notti"]=booking_roma.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
booking_roma["numero_persone"]=booking_roma.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_roma["inizio_permanenza"]=booking_roma.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_roma["fine_permanenza"]=booking_roma.date.str.split("-").str[1]

# Zona, estratta dalla variabile indirizzo.
booking_roma["zona"] = booking_roma.indirizzo.str.split(",").str[0]

# Città, estratta dalla variabile indirizzo.
booking_roma["città"] = booking_roma.indirizzo.str.split(",").str[1]

In [101]:
booking_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,zona,città
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,1 notte,2 adulti,1 agosto,2 agosto,Trionfale,Roma


# Data Cleaning

## Riodinare le variabili

Solo per maggiore ordine e chiarezza riordino le variabili.

In [102]:
agoda_roma = agoda_roma[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo',
'valutazione']]

agoda_terni = agoda_terni[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo',
'valutazione']]

booking_roma = booking_roma[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo','valutazione_booking','stelle',  'descrizione_camera',
       'configurazione_camera', 'descrizione_unprocessed']]

booking_terni = booking_terni[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo','valutazione_booking','stelle',  'descrizione_camera',
       'configurazione_camera', 'descrizione_unprocessed']]

## Cleaning dei dati riguardanti Agoda

Come prima cosa si uniscono i datataset **agoda_roma** e **agoda_terni** così da lavorare su un solo dataset.

In [103]:
agoda = pd.concat([agoda_roma,agoda_terni]) # Unisco agoda_roma e agoda_terni
agoda.reset_index(inplace=True,drop=True) # Nuovi indici, Eliminare quelli precedenti

In [104]:
display(agoda.head(1))
display(agoda.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione
0,Raeli Hotel Lazio,Stazione centrale Roma Termini,Roma,In pieno centro,€159,1 notte,2 adulti,1 agosto,2 agosto,"7,7",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione
676,Casa vacanze TERRA DI ASLAN,Narni,Narni,NaN,119 €,1 notte,2 adulti,1 agosto,2 agosto,"9,6",Eccezionale,7 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti","Narni, Narni",3.0


In [105]:
# Pulizia della variabile 'prezzo' : rimosso simbolo euro ed il punto separatore delle migliaia tramite la funzione pulisci_prezzo, variabile trasformata da object a int.
agoda.prezzo = pulisci_prezzo(agoda.prezzo)

# Pulizia della variabile 'numero di recensioni' -> rimuove la parola 'recensioni, Se la recensione risulta 'novità su agoda' assegna 0. rimuove anche il separatore delle migliaia. Discutible l'assegnazione di 0 recensioni quando è una novita.
agoda.numero_recensioni =  agoda.numero_recensioni.apply(lambda row: pulisci_recensione(row))

# Pulizia della variabile 'distanza_centro' -> Lascia solo la distanza in km.
agoda["distanza_centro"] =  agoda.distanza_centro.apply(lambda row: pulisci_distanza_centro(row))

# Rendere la variabile numero di persone un intero ed eliminare la parola 'adulti'
agoda["numero_persone"]=agoda.numero_persone.str.replace("adulti","").astype(int)

# Rendere la variabile numero di notti un intero ed eliminare la parola 'notte'
agoda["numero_notti"]=agoda.numero_notti.str.replace("notte","").astype(int)

# Pulizia della variabile 'inizio_permanenza' -> Trasforma la data in formato italiano.
agoda["inizio_permanenza"] =  agoda.inizio_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
agoda["inizio_permanenza_datetime"]=pd.to_datetime(agoda["inizio_permanenza"])

# Pulizia della variabile 'fine_permanenza' -> -> Trasforma la data in formato italiano.
agoda["fine_permanenza"] =  agoda.fine_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
agoda["fine_permanenza_datetime"]=pd.to_datetime(agoda["fine_permanenza"])


In [106]:
display(agoda.head(1))
display(agoda.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
0,Raeli Hotel Lazio,Stazione centrale Roma Termini,Roma,0.0,159,1,2,01-08-25,02-08-25,"7,7",Ottimo,321,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
676,Casa vacanze TERRA DI ASLAN,Narni,Narni,NaN,119,1,2,01-08-25,02-08-25,"9,6",Eccezionale,7,1 agosto - 2 agosto,"1 notte, 2 adulti","Narni, Narni",3.0,2025-01-08,2025-02-08


In [107]:
agoda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   titolo                      677 non-null    object        
 1   zona                        677 non-null    object        
 2   città                       677 non-null    object        
 3   distanza_centro             539 non-null    float64       
 4   prezzo                      677 non-null    int64         
 5   numero_notti                677 non-null    int64         
 6   numero_persone              677 non-null    int64         
 7   inizio_permanenza           677 non-null    object        
 8   fine_permanenza             677 non-null    object        
 9   recensione_voto_numerico    677 non-null    object        
 10  recensione_voto_parola      677 non-null    object        
 11  numero_recensioni           677 non-null    int64         

## Cleaning dei dati riguardanti Booking

Come prima cosa si uniscono i datataset **booking_roma** e **booking_terni** così da lavorare su un solo dataset.

In [108]:
booking = pd.concat([booking_roma,booking_terni]) # Unisco agoda_roma e agoda_terni
booking.reset_index(inplace=True,drop=True) # Nuovi indici, Eliminare quelli precedenti
display(booking.head(1))
display(booking.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,Trionfale,Roma,"3,5 km dal centro",€ 242,1 notte,2 adulti,1 agosto,2 agosto,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed
885,-15min Marmore- Ampio Appartamento Terra e Cuore,Terni,Terni,"2,9 km dal centro",€ 182,1 notte,2 adulti,1 agosto,2 agosto,Novità su booking,Novità su booking,Novità su booking,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,NaN,NaN,Appartamento con 2 Camere da Letto,Configurazione camera non disponibile,Appartamento con 2 Camere da Letto4 letti (2 s...


In [109]:
# Pulizia della variabile 'prezzo' : rimosso simbolo euro ed il punto separatore delle migliaia tramite la funzione pulisci_prezzo, variabile trasformata da object a int.
booking.prezzo = pulisci_prezzo(booking.prezzo)

# Pulizia della variabile 'numero di recensioni' -> rimuove la parola 'recensioni, Se la recensione risulta 'novità su agoda' assegna 0. rimuove anche il separatore delle migliaia. Discutible l'assegnazione di 0 recensioni quando è una novita.
booking.numero_recensioni =  booking.numero_recensioni.apply(lambda row: pulisci_recensione(row))

# Pulizia della variabile 'distanza_centro' -> Lascia solo la distanza in km.
booking["distanza_centro"] =  booking.distanza_centro.apply(lambda row: pulisci_distanza_centro(row))

# Pulizia variabili: recensione_voto_parola, recensione_voto_parola mettere NAN dove è specificato 'Novità su booking'
booking.loc[booking.recensione_voto_numerico == "Novità su booking","recensione_voto_numerico"] = np.nan # Questo è il formato giusto per assegnare un nuovo valore ad una variabile nel dataframe che rispetta alcune condizioni
booking.loc[booking.recensione_voto_parola == "Novità su booking","recensione_voto_parola"] = np.nan

# Rendere la variabile numero di persone un intero ed eliminare la parola 'adulti'
booking["numero_persone"]=booking.numero_persone.str.replace("adulti","").astype(int)

# Rendere la variabile numero di notti un intero ed eliminare la parola 'notte'
booking["numero_notti"]=booking.numero_notti.str.replace("notte","").astype(int)

# Pulizia della variabile 'inizio_permanenza' -> Trasforma la data in formato italiano.
booking["inizio_permanenza"] =  booking.inizio_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
booking["inizio_permanenza_datetime"]=pd.to_datetime(booking["inizio_permanenza"])

# Pulizia della variabile 'fine_permanenza' -> -> Trasforma la data in formato italiano.
booking["fine_permanenza"] =  booking.fine_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
booking["fine_permanenza_datetime"]=pd.to_datetime(booking["fine_permanenza"])

In [110]:
display(booking.head(1))
display(booking.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
0,The Sereno-3,Trionfale,Roma,3.5,242,1,2,01-08-25,02-08-25,"8,1",Ottimo,10,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,2025-01-08,2025-02-08


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
885,-15min Marmore- Ampio Appartamento Terra e Cuore,Terni,Terni,2.9,182,1,2,01-08-25,02-08-25,NaN,NaN,0,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,NaN,NaN,Appartamento con 2 Camere da Letto,Configurazione camera non disponibile,Appartamento con 2 Camere da Letto4 letti (2 s...,2025-01-08,2025-02-08


In [111]:
booking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   titolo                      886 non-null    object        
 1   zona                        886 non-null    object        
 2   città                       861 non-null    object        
 3   distanza_centro             886 non-null    float64       
 4   prezzo                      886 non-null    int64         
 5   numero_notti                886 non-null    int64         
 6   numero_persone              886 non-null    int64         
 7   inizio_permanenza           886 non-null    object        
 8   fine_permanenza             886 non-null    object        
 9   recensione_voto_numerico    881 non-null    object        
 10  recensione_voto_parola      881 non-null    object        
 11  numero_recensioni           886 non-null    int64         

# Preparazione dati per il record linkage

Queste ulteriori trasformazioni sono mirate all'ottimizzazione del formato dei dati in vista del record linkage.     
Ovvero manipolare le variabili:
   - titolo
   - zona

In modo da facilitare il più possibile le operazioni di record linkage, sia in velocità che in accuratezza.

## Titolo

### Creazione nuova variabile 

Creare una nuova variabile dal titolo chiamata **titolo_processed**.    
Questa variabile prende il titolo e applica diverse trasformazioni.
   - Rende il testo minuscolo;
   - lo ordina
   - rimuove caratteri speciali
In questo modo si riusciranno ad identificare più facilmente e velocemente gli stessi hotel e B&B. 


In [112]:
# Creazione nuova variabile partendo dal titolo, il titolo viene mezzo in minuscolo, rimossi caratteri speciali ed ordinato.
agoda.insert(loc = 1,
             column = 'titolo_processed',
             value = agoda.titolo.apply(lambda x: alfabeto(x)))

# Creazione nuova variabile partendo dal titolo, il titolo viene mezzo in minuscolo, rimossi caratteri speciali ed ordinato.
booking.insert(loc = 1,
               column = 'titolo_processed',
               value = booking.titolo.apply(lambda x: alfabeto(x)))

In [113]:
display(agoda.head(1))
display(booking.head(1))

,titolo,titolo_processed,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
0,Raeli Hotel Lazio,Hotel Lazio Raeli,Stazione centrale Roma Termini,Roma,0.0,159,1,2,01-08-25,02-08-25,"7,7",Ottimo,321,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08


,titolo,titolo_processed,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
0,The Sereno-3,3 Sereno The,Trionfale,Roma,3.5,242,1,2,01-08-25,02-08-25,"8,1",Ottimo,10,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,2025-01-08,2025-02-08


### Pulizia della nuova variabile

Pulizia titolo da parole roma e rome

In [114]:
# Pulizia titolo
agoda['titolo_processed'] = (
    agoda['titolo_processed']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r',?\s*rome\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.strip()  # rimuove spazi iniziali/finali
    .str.title()
)

# Pulizia titolo
booking['titolo_processed'] = (
    booking['titolo_processed']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r',?\s*rome\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.strip()  # rimuove spazi iniziali/finali
    .str.title() 
    )

## Zona

Nuova variabile **zona_processed**, rimuovere le parole più comuni dalla zona come:
   - città
   - piazza
   - rione
...

In [115]:
agoda.insert(loc = 3,
             column = 'zona_processed',
             value = agoda['zona']
                        .str.lower()  # tutto minuscolo
                        .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
                        .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
                        .str.replace(r'\bcentrale\b', '', regex=True)  # rimuove "centrale" come parola intera
                        .str.replace(r'\b(rione|piazza|fontana|di|del|e|città|citta|citt)\b', '', regex=True)  # rimuove parole specificate
                        .str.replace(r'\s+', ' ', regex=True)  # rimuove spazi multipli eventualmente lasciati
                        .str.strip()  # rimuove spazi iniziali/finali
            )

booking.insert(loc = 3,
                column = 'zona_processed',
                value = booking['zona']
                        .str.lower()  # tutto minuscolo
                        .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
                        .str.replace(r'\bcentrale\b', '', regex=True)  # rimuove "centrale" come parola intera
                        .str.replace(r'\b(rione|piazza|fontana|di|del|e|città|citta|citt)\b', '', regex=True)  # rimuove parole specificate
                        .str.replace(r'\s+', ' ', regex=True)  # rimuove spazi multipli eventualmente lasciati
                        .str.strip()
            )

In [116]:
display(agoda.head(1))
display(booking.head(1))

,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
0,Raeli Hotel Lazio,Hotel Lazio Raeli,Stazione centrale Roma Termini,stazione termini,Roma,0.0,159,1,2,01-08-25,02-08-25,"7,7",Ottimo,321,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08


,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
0,The Sereno-3,3 Sereno The,Trionfale,trionfale,Roma,3.5,242,1,2,01-08-25,02-08-25,"8,1",Ottimo,10,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,2025-01-08,2025-02-08


## Città

 Pulizia titoli, si utilizza la funzione clean del pacchetto record linkage.
 In particolare, clean() fa le seguenti operazioni:
  - Rende il testo minuscolo (lowercase).
  - Rimuove punteggiatura, simboli e caratteri speciali.
  - Rimuove spazi in eccesso (trimma e normalizza spazi multipli).
  - Sostituisce caratteri accentati con equivalenti ASCII (es. "é" → "e").

In [117]:
agoda['città'] = clean(agoda['città'])           # Pulizia tramite la funzione 'clean' del pacchetto recordlinkage
booking['città'] = clean(booking['città'])       # Pulizia tramite la funzione 'clean' del pacchetto recordlinkage

# AGGIUNTA FAKE RECORD

Inserire un **hotel del valentino** (presente a Terni) nella città di Roma.

In [118]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_processed': 'hotel valentino', 'prezzo': 190 ,'zona_processed':'eur','città':'Roma'}])

# Aggiunta al dataset
agoda = pd.concat([agoda, nuova_riga], ignore_index=True)

In [119]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_processed': 'paris hotel', 'prezzo': 110 ,'zona_processed':'termini','città':'Roma'}])

# Aggiunta al dataset
agoda = pd.concat([agoda, nuova_riga], ignore_index=True)

In [120]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_processed': 'fiori hotel', 'prezzo': 81 ,'zona_processed':'Terni','città':'Terni'}])

# Aggiunta al dataset
agoda = pd.concat([agoda, nuova_riga], ignore_index=True)

Inserire un errore battitura, **classic hotel Tulipano** diventa **classic hotel Tulippano**, sarà utile per capire se verranno matchati correttamente.

In [121]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_processed': 'classic hotel tulippano', 'prezzo': 109 ,'zona_processed':'Roma','città':'Roma'}])

# Aggiunta al dataset
agoda = pd.concat([agoda, nuova_riga], ignore_index=True)

In [122]:
# Visualizzare i fake records
agoda.tail()

,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
676,Casa vacanze TERRA DI ASLAN,Aslan Casa Di Terra Vacanze,Narni,narni,narni,NaN,119,1.0,2.0,01-08-25,02-08-25,"9,6",Eccezionale,7.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Narni, Narni",3.0,2025-01-08,2025-02-08
677,NaN,hotel valentino,NaN,eur,Roma,NaN,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
678,NaN,paris hotel,NaN,termini,Roma,NaN,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
679,NaN,fiori hotel,NaN,Terni,Terni,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
680,NaN,classic hotel tulippano,NaN,Roma,Roma,NaN,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


# Esportazione dati

In [123]:
agoda.to_csv(f"{path_data_product}/agoda.csv")

In [124]:
booking.to_csv(f"{path_data_product}/booking.csv")

In [125]:
agoda

,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
0,Raeli Hotel Lazio,Hotel Lazio Raeli,Stazione centrale Roma Termini,stazione termini,roma,0.0,159,1.0,2.0,01-08-25,02-08-25,"7,7",Ottimo,321.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08
1,Raeli Hotel Noto,Hotel Noto Raeli,Stazione centrale Roma Termini,stazione termini,roma,0.0,153,1.0,2.0,01-08-25,02-08-25,"7,8",Ottimo,2391.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08
2,Hotel King,Hotel King,Via Veneto,via veneto,roma,0.0,169,1.0,2.0,01-08-25,02-08-25,"8,1",Fantastico,2109.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Via Veneto, Roma - In pieno centro",3.0,2025-01-08,2025-02-08
3,Rome Cavalieri A Waldorf Astoria Hotel,A Astoria Cavalieri Hotel Waldorf,Trionfale,trionfale,roma,1.2,1620,1.0,2.0,01-08-25,02-08-25,"8,9",Fantastico,1161.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma - A 1,2 km dal centro",5.0,2025-01-08,2025-02-08
4,Hotel Taormina,Hotel Taormina,Stazione centrale Roma Termini,stazione termini,roma,0.0,105,1.0,2.0,01-08-25,02-08-25,"8,3",Fantastico,1541.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",2.0,2025-01-08,2025-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,Casa vacanze TERRA DI ASLAN,Aslan Casa Di Terra Vacanze,Narni,narni,narni,NaN,119,1.0,2.0,01-08-25,02-08-25,"9,6",Eccezionale,7.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Narni, Narni",3.0,2025-01-08,2025-02-08
677,NaN,hotel valentino,NaN,eur,Roma,NaN,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
678,NaN,paris hotel,NaN,termini,Roma,NaN,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
679,NaN,fiori hotel,NaN,Terni,Terni,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [126]:
booking

,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
0,The Sereno-3,3 Sereno The,Trionfale,trionfale,roma,3.50,242,1,2,01-08-25,02-08-25,"8,1",Ottimo,10,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,2025-01-08,2025-02-08
1,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Apartments Bonvoy By Marriott Piazza Sonder Ve...,Pantheon,pantheon,roma,0.15,322,1,2,01-08-25,02-08-25,"8,1",Ottimo,321,1 agosto - 2 agosto,"1 notte, 2 adulti","Pantheon, Roma",NaN,3.0,Camera Queen,Configurazione camera non disponibile,Camera Queen 1 letto matrimonialeResta solo 1 ...,2025-01-08,2025-02-08
2,GuestHouse Martin,Guesthouse Martin,Villa Borghese/Parioli,villa borghese/parioli,roma,3.30,69,1,2,01-08-25,02-08-25,"8,8",Favoloso,267,1 agosto - 2 agosto,"1 notte, 2 adulti","Villa Borghese/Parioli, Roma",4.0,NaN,Camera Doppia con Letti Singoli con Bagno Priv...,Configurazione camera non disponibile,Camera Doppia con Letti Singoli con Bagno Priv...,2025-01-08,2025-02-08
3,ASTORIA GOLDEN GATE,Astoria Gate Golden,Stazione Termini,stazione termini,roma,2.00,110,1,2,01-08-25,02-08-25,"8,9",Favoloso,1446,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione Termini, Roma",NaN,2.0,Camera Matrimoniale Superior,Configurazione camera non disponibile,Camera Matrimoniale Superior Vari tipi di lett...,2025-01-08,2025-02-08
4,Magnifico Rome,Magnifico,Stazione Termini,stazione termini,roma,1.10,107,1,2,01-08-25,02-08-25,"7,8",Buono,781,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione Termini, Roma",4.0,NaN,Camera Matrimoniale Economy,Configurazione camera non disponibile,Camera Matrimoniale Economy1 letto matrimonial...,2025-01-08,2025-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,Cuore di Terni,Cuore Di Terni,Terni,terni,terni,0.80,96,1,2,01-08-25,02-08-25,NaN,NaN,0,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,NaN,NaN,Appartamento con 2 Camere da Letto,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da LettoIntero appar...,2025-01-08,2025-02-08
882,Frantoio Pistelli,Frantoio Pistelli,Terni,terni,terni,2.00,81,1,2,01-08-25,02-08-25,"9,0",Eccellente,6,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,NaN,NaN,Appartamento con 1 Camera da Letto,Intero appartamento • 1 camera da letto • 1 zo...,Appartamento con 1 Camera da LettoIntero appar...,2025-01-08,2025-02-08
883,"Favolosa ""Dimora del Capriolo"" con Giardino e ...",Capriolo Con Del Dimora E Favolosa Giardino Pa...,Terni,terni,terni,2.90,216,1,2,01-08-25,02-08-25,"9,3",Eccellente,13,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,3.0,NaN,Appartamento con 1 Camera da Letto,Intero appartamento • 1 camera da letto • 1 zo...,Appartamento con 1 Camera da LettoIntero appar...,2025-01-08,2025-02-08
884,La Severa 2,2 La Severa,Terni,terni,terni,1.40,96,1,2,01-08-25,02-08-25,"8,4",Ottimo,11,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,4.0,NaN,Appartamento con 1 Camera da Letto,Intero appartamento • 1 camera da letto • 1 zo...,Appartamento con 1 Camera da LettoIntero appar...,2025-01-08,2025-02-08
